<a href="https://colab.research.google.com/github/ekaratnida/Applied-machine-learning/blob/master/Week08-workshop-2/Week8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Import library

In [88]:

import pickle
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)
import warnings
warnings.filterwarnings('ignore')

2. Load data and partition X,y

In [89]:
data = pd.read_csv('https://raw.githubusercontent.com/ekaratnida/Applied-machine-learning/master/Week04-workshop-1/data.txt')
print(data.head())
#X = data[["area","rooms"]]
X = data.iloc[:,0:2]
print(X.head())

#y = data.price
#y = data.iloc[:,2]
y = data[["price"]] 
print(y.head())

   area  rooms   price
0  2104      3  399900
1  1600      3  329900
2  2400      3  369000
3  1416      2  232000
4  3000      4  539900
   area  rooms
0  2104      3
1  1600      3
2  2400      3
3  1416      2
4  3000      4
    price
0  399900
1  329900
2  369000
3  232000
4  539900


3. Train model

3.1 Train/test split

In [90]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

3.2 Cross validation </br>
GridSearchCV will split X into two datasets, i.e., train and validation set.

In [91]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree), LinearRegression(**kwargs))

param_grid = {'polynomialfeatures__degree': np.arange(5), 'linearregression__fit_intercept': [True, False], 'linearregression__normalize': [True, False]}
poly_grid = GridSearchCV(PolynomialRegression(), param_grid, cv=5, scoring='neg_mean_squared_error', verbose=0)
poly_grid.fit(X,y)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('polynomialfeatures',
                                        PolynomialFeatures()),
                                       ('linearregression',
                                        LinearRegression())]),
             param_grid={'linearregression__fit_intercept': [True, False],
                         'linearregression__normalize': [True, False],
                         'polynomialfeatures__degree': array([0, 1, 2, 3, 4])},
             scoring='neg_mean_squared_error')

In [92]:
print(poly_grid.best_params_)

{'linearregression__fit_intercept': True, 'linearregression__normalize': True, 'polynomialfeatures__degree': 1}


4. Model evaluation

In [93]:
from sklearn.metrics import mean_squared_error
y_pred = poly_grid.predict(X_test)
print(y_test, " ", y_pred)
mseValues = mean_squared_error(y_test,y_pred)
print(mseValues)

     price
27  469000
39  287000
26  464500
43  299000
24  573900
36  229900
12  329999
19  599000
4   539900
25  249900
8   212000
3   232000
6   314900
40  368500
33  579900   [[415030.01477434]
 [355619.31031959]
 [266353.01492351]
 [230436.66102696]
 [604913.37413438]
 [235865.87731365]
 [326492.02609913]
 [476593.38604091]
 [472277.85514636]
 [216515.5936252 ]
 [255494.58235014]
 [269244.1857271 ]
 [276933.02614885]
 [303768.43288347]
 [500087.73659911]]
5903031511.737383


4. Predict new data

In [94]:
X_new = pd.DataFrame(
    {
    "area":[2000],
    "rooms":[6]
    }
)
y_new = poly_grid.predict(X_new)
print(y_new)

[[315591.14290408]]


5. Deploy

In [95]:
'''print("Deploy")
pickle.dump(poly_grid, open('model.sav','wb') )
from google.colab import files
files.download('model.sav')'''

'print("Deploy")\npickle.dump(poly_grid, open(\'model.sav\',\'wb\') )\nfrom google.colab import files\nfiles.download(\'model.sav\')'